In [16]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import sys
import re
import json

from pyspark.ml import Pipeline


In [30]:
class Loader:
    def __init__(self, rootDir):
        self.rootDir = rootDir
        
    def loadData(self, sparkContext):
        pass

class DataLoader(Loader):
    key = 'key'
    content = 'content'
    path = 'path'
    
    def loadData(self, sc):
        def extractKey(entry):
            fpath, content = entry
            fname = fpath.split('/')[-1]
            key = re.sub('\.json$', '', fname)
            return (key, json.loads(content)) #json parsing should be part of transformer code
        
        mappings = sc.textFile(self.rootDir + 'mapping.csv')        \
            .map(lambda line: line.split('|'))                      \
            .toDF([self.path, self.key])
        
        statements = sc.wholeTextFiles(self.rootDir + 'statement/') \
            .map(extractKey)                                        \
            .toDF([self.key, self.content])
    
        return mappings.join(statements, 'key')

### Zaimportowanie modułów przygotowanych przez drugą grupę

In [18]:
from post_extractor.modules.posts import *

### *Definicja źródła danych (przykładowa klasa)*

In [32]:
source = DataLoader('./kascysko.blogspot.com/')

### *Defnicja przepływu*
Przepływ definiowany jest poprzez stworzenie instancji kolejnych metod i przekazanie im kolumny (bądź kolumn) wejściowej i specyfikacja wyjścia każdej z metod. Następnie tworzony jest obiekt `pipeline` grupujący kolejne transformery w przepływ. Istotne jest aby zachować odpowiednią kolejność podczas przekazywania metod do pipeline, powinna ona być zgodna z zależnościami między metodami.

In [34]:
poster = PostTransformer()                  \
    .setInputCol(source.content)            \
    .setOutputCol('posts')

translator = TranslateTransformer()         \
    .setInputCol(poster.getOutputCol())     \
    .setOutputCol('translated')
        
sentencer = SentenceTransformer()           \
    .setInputCol(translator.getOutputCol()) \
    .setOutputCol('sentences')              

pipeline = Pipeline(stages=[poster, translator, sentencer])

### Setup - uruchomienie Sparka oraz odczyt danych z dysku

In [38]:
sconf = SparkConf()              \
    .setMaster('local[*]')       \
    .setAppName('PipelineFlow')

sc = SparkContext.getOrCreate(sconf)
df = source.loadData(sc)

### Uruchomienie przepływu i odczyt danych
Po dopasowaniu parametrów metod składających się na przepływ otrzymujemy model, który może być później zapisany dzieki czemu następnym razem nie będzie konieczności dobierania parametrów dla nowych danych.

In [40]:
pipelineModel = pipeline.fit(df)
output = pipelineModel.transform(df)

In [41]:
output.first().sentences[:5]

['I Jantar spiced with yeast beer :) described wcierka for me a bit too expensive',
 'I have banyan water with isany, but on holiday I am not very well with regularity.',
 'Jantar is waiting for autumn.',
 'I have this wcierkę on wishliscie but is piekielnie droga: /',
 'I like to use all wcierki :)']

### Zmiana przepływu

In [35]:
pos_counter = SpeechPartsTransformer()       \
    .setInputCol(translator.getOutputCol())  \
    .setOutputCol('pos_count')

pipeline_alt = Pipeline(stages=[poster, translator, pos_counter])

In [36]:
out_alt = pipeline_alt.fit(df).transform(df)


In [37]:
out_alt.first().pos_count

{'CC': 28,
 'CD': 6,
 'DT': 69,
 'EX': 2,
 'IN': 107,
 'JJ': 57,
 'JJR': 3,
 'JJS': 2,
 'MD': 19,
 'NN': 138,
 'NNP': 23,
 'NNS': 32,
 'PDT': 1,
 'PRP': 105,
 'PRP$': 12,
 'RB': 66,
 'RBR': 3,
 'RBS': 1,
 'SYM': 1,
 'TO': 13,
 'UH': 2,
 'VB': 40,
 'VBD': 10,
 'VBG': 12,
 'VBN': 8,
 'VBP': 49,
 'VBZ': 32,
 'WP': 10,
 'WRB': 5}